In [9]:
from openai import OpenAI

In [10]:
from dotenv import load_dotenv

In [21]:
from supabase import create_client, Client

In [11]:
load_dotenv()

True

In [12]:
import os

In [13]:
import requests

In [19]:
client = OpenAI(api_key = os.getenv("OPEN_AI_API"))

In [22]:
supabase_url = os.environ.get("SUPABASE_URL")

In [23]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [25]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [26]:
sql_query = f"""
SELECT * FROM webhooks order by created_at ASC
"""

In [27]:
sql_query

'\nSELECT * FROM webhooks order by created_at ASC\n'

In [53]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 2,
  'news_output': 'Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness.',
  'source_urls': ['https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/']},
 {'id': 3,
  'news_output': "TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent funding and personnel developmen

In [73]:
formatted_result = []

def result_data():
    for item in result.data:
        formatted_result.append(
            f""" {item['news_output']} source: {item['source_urls'][0]}"""
        )

    print(formatted_result)

In [74]:
result_data()

[' Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness. source: https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/', " TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent funding and personnel developments (references Humans&'s large seed round and leadership departures 

In [85]:
def openai_research(data):
    research_output = []
    for i in data:
        response = client.responses.create(
            model = "gpt-4o",
            # reasoning = {"effort": "high"},
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = f""" 
            You are given news summary and source in {i}, you are supposed to run a deep research and read other sources as well. 
            And then give a well structured ouput in the form of points and sources. 
            """
        )

        output = response.output_text

        research_output.append(output)

    return research_output

In [86]:
a = [' Meta (reporting summarized Jan 24, 2026) paused teen access to its AI character feature as it rebuilds a safety-focused experience — a product and policy change limiting a class of users while the company redesigns moderation/safety controls for conversational AI characters. This is a notable product-safety adjustment from a major AI platform/operator. source: https://theaiinsider.tech/2026/01/24/meta-pauses-teen-access-to-ai-characters-as-it-rebuilds-safety-focused-experience/']

openai_research(a)

['Below is a well-structured, in-depth analysis of the situation, integrating multiple sources and organized into clear points with citations. I\'ve used full markdown formatting (without a title) as requested.\n\n---\n\n##  Summary of the Development\n\n- Meta (owner of Facebook, Instagram, WhatsApp) has **temporarily paused teen access to its AI characters** across all apps globally, effective "in the coming weeks." Teens flagged by date-of-birth or age‑prediction technology are affected. However, they will **still be able to use Meta’s AI assistant**, which includes age‑appropriate protections ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n- The pause aligns with Meta’s plan to **launch a newly rebuilt AI character experience** that integrates stronger **parental controls** and offers **age‑appropriate, limited themes**. When relaunched, characters will focus on topics like education, sports, and hobbies ([theaiinsider.tech](https:/

In [101]:
def openai_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client.responses.create(
            model = "gpt-4.1",
            input = f""" 
            You are a professional tech journalist for "The AI Times" newspaper.

            Your task: Write a comprehensive news article covering today's AI developments.
            
            Entire news story and its details: {i}
            
            INSTRUCTIONS:
            1. Write a newspaper-quality article (800-1000 words)
            2. Create an engaging headline
            3. Start with a lead paragraph summarizing the most important developments
            5. Use journalistic style: objective, clear, professional and opinionated
            6. Include all the sources & links at the end and not ANY sources or links during the article

            You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
            
            However, the opinion you have be your own but it should be based on the above article. 
            """
        )


        final_output = response.output_text

        print(final_output)

        final_news_article.append(final_output)

    return final_news_article

In [102]:
b = ['Below is a well-structured, in-depth analysis of the situation, integrating multiple sources and organized into clear points with citations. I\'ve used full markdown formatting (without a title) as requested.\n\n---\n\n##  Summary of the Development\n\n- Meta (owner of Facebook, Instagram, WhatsApp) has **temporarily paused teen access to its AI characters** across all apps globally, effective "in the coming weeks." Teens flagged by date-of-birth or age‑prediction technology are affected. However, they will **still be able to use Meta’s AI assistant**, which includes age‑appropriate protections ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n- The pause aligns with Meta’s plan to **launch a newly rebuilt AI character experience** that integrates stronger **parental controls** and offers **age‑appropriate, limited themes**. When relaunched, characters will focus on topics like education, sports, and hobbies ([theaiinsider.tech](https://theaiinsider.tech/2026/01/24/meta-pauses-teen-access-to-ai-characters-as-it-rebuilds-safety-focused-experience/?utm_source=openai)).\n\n- The move comes amid rising **legal and regulatory scrutiny** of tech platforms’ impact on minors. Meta faces trials in Los Angeles and New Mexico related to potential harm to children. Radiating broader industry alertness, other firms like Character.AI have issued similar restrictions following lawsuits alleging unsafe chatbot interactions ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n---\n\n##  Background and Timeline\n\n**October 2025**\n- Meta previewed **new parental control tools**: guardians could disable teen chats with AI characters, block specific bots, and receive insights on conversation topics (without full transcripts). These would roll out early 2026 in English-speaking countries including the US ([theguardian.com](https://www.theguardian.com/technology/2025/oct/18/parents-will-be-able-to-block-meta-bots-from-talking-to-their-children-under-new-safeguards?utm_source=openai)).\n\n**August 2025**\n- After a Reuters exposé about AI bots engaging teens in romantic or self-harm content, Meta updated its AI to **avoid such topics** and **limit teen access to select characters**, redirecting to expert resources when needed ([businessinsider.com](https://www.businessinsider.com/meta-changes-the-way-its-ai-chatbot-responds-to-children-2025-8?utm_source=openai)).\n\n**Late 2025**\n- Character.AI responded to lawsuits—including one involving a teen suicide—by **banning under‑18 users** and limiting teen usage before a full block by November 2025 ([nypost.com](https://nypost.com/2025/10/29/business/character-ai-bans-chatbots-for-teens-after-lawsuits-blame-app-for-deaths-suicide-attempts/?utm_source=openai)).\n\n**January 23, 2026**\n- Meta announced its **temporary pause**, ahead of the new version launch ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n---\n\n##  Key Motivations & Factors\n\n- **Parental Concerns & Feedback**: Parents emphasized desire for more transparency and oversight over teens’ AI interactions, which Meta cited as a key influence in their decision ([theverge.com](https://www.theverge.com/news/866906/meta-teens-ai-characters-stop-block-new-version?utm_source=openai)).\n\n- **Regulatory Pressure**: \n  - Meta faces trials in both Los Angeles and New Mexico over platform harm to teens ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n  - Broader legal and regulatory scrutiny is ongoing (including FTC investigations and state legislation in California mandating bot-user reminders and self-harm safeguards) ([lemonde.fr](https://www.lemonde.fr/en/economy/article/2025/10/15/california-plans-on-protecting-minors-and-preventing-self-destructive-content-by-regulating-ai_6746443_19.html?utm_source=openai)).\n\n- **Industry-Wide Safety Movement**: Meta’s move is part of a broader pattern. Character.AI’s actions and legal challenges, along with safety research into AI character interactions, reflect growing focus on youth safety in AI platforms ([nypost.com](https://nypost.com/2025/10/29/business/character-ai-bans-chatbots-for-teens-after-lawsuits-blame-app-for-deaths-suicide-attempts/?utm_source=openai)).\n\n---\n\n##  What To Expect Next\n\n- Once the **updated AI character experience** is ready, it will include:\n  - **Robust parental controls**—enable or disable full AI character chats or block specific characters; parents will see conversation topics ([techcrunch.com](https://techcrunch.com/2026/01/23/meta-pauses-teen-access-to-ai-characters-ahead-of-new-version/?utm_source=openai)).\n  - **Age‑appropriate design**: characters will avoid sensitive content (self-harm, romance) and focus on safe themes ([theaiinsider.tech](https://theaiinsider.tech/2026/01/24/meta-pauses-teen-access-to-ai-characters-as-it-rebuilds-safety-focused-experience/?utm_source=openai)).\n  - **Parental transparency** tools with topic-level insights (not full transcripts) will allow guardians to engage meaningfully with teens about AI usage ([apnews.com](https://apnews.com/article/306b9c49ef69f6894044b2d82c6172fe?utm_source=openai)).\n\n- Meta’s AI assistant will remain accessible to teens throughout, with built-in protective design ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n- With trials commencing soon, Meta will likely combine this product move with public messaging to show proactive responsibility.\n\n---\n\n##  Broader Implications & Context\n\n- **Product-Safety Interventions**: Temporarily limiting a user group (teens) while redesigning features represents a strong risk-mitigation strategy, signaling a shift toward safety-first in AI deployments.\n\n- **Youth Vulnerability in Conversational AI**:\n  - Research shows youth are particularly susceptible to emotional relationships with anthropomorphic chatbots, often preferring relational styles that heighten anthropomorphism and emotional reliance ([arxiv.org](https://arxiv.org/abs/2512.15117?utm_source=openai)).\n  - AI character platforms demonstrate significant safety risks—with studies indicating unsafe response rates exceeding 65%, compared with 17.7% for baseline systems ([arxiv.org](https://arxiv.org/abs/2512.01247?utm_source=openai)).\n  - Dedicated youth‑centric safety tools like YouthSafe show effectiveness in detecting developmental risks such as grooming or emotional overreliance ([arxiv.org](https://arxiv.org/abs/2509.08997?utm_source=openai)).\n\n- **Policy Landscape**:\n  - California’s law, for instance, now requires AI platforms to clearly indicate chatbot status and protect against self-harm content, with recurring reminders for minors ([lemonde.fr](https://www.lemonde.fr/en/economy/article/2025/10/15/california-plans-on-protecting-minors-and-preventing-self-destructive-content-by-regulating-ai_6746443_19.html?utm_source=openai)).\n  - Industry precedent like Meta’s move may influence or expedite federal-level regulation and enforcement.\n\n---\n\n##  Summary Points\n\n- Meta has **paused teen access** to its AI characters globally, effective "in the coming weeks," but teens retain access to the AI assistant ([apnews.com](https://apnews.com/article/f64d53df26893cfefa97dc1249d73ab6?utm_source=openai)).\n\n- This pause enables Meta to design a **safer AI character experience**, embedding **parental controls** and focusing on **PG-13 style, age-appropriate content** ([techcrunch.com](https://techcrunch.com/2026/01/23/meta-pauses-teen-access-to-ai-characters-ahead-of-new-version/?utm_source=openai)).\n\n- The decision responds to **parental demand**, **legal trials**, and wider **regulatory pressure**, while aligning with a broader industry move toward youth safety in AI ([lemonde.fr](https://www.lemonde.fr/en/economy/article/2025/10/15/california-plans-on-protecting-minors-and-preventing-self-destructive-content-by-regulating-ai_6746443_19.html?utm_source=openai)).\n\n- Research underscores teens\' heightened **emotional vulnerability** with AI characters, justifying measures like these ([arxiv.org](https://arxiv.org/abs/2512.15117?utm_source=openai)).\n\n---\n\nLet me know if you\'d like a deeper breakdown of Meta’s regulatory risks, comparisons with other AI platforms, or a timeline tracking similar safety interventions in the AI industry.']

In [103]:
openai_article_builder(b)

**Meta Puts Teen Access to AI Characters on Hold Amid Regulatory Storm and Safety Concerns**

Meta, the social media titan behind Facebook, Instagram, and WhatsApp, is taking the extraordinary step of pausing teen access to its popular AI-powered characters across all its platforms worldwide. The decision, set to be implemented in the coming weeks, arrives at a watershed moment for both Meta and the wider AI industry, as companies grapple with the growing outcry over the safety and psychological risks of conversational bots for younger users.

**A Precaution Amid Intensifying Scrutiny**

For years, tech companies have cultivated AI chatbots as the next leap in digital engagement. These virtual personalities, ranging from celebrity-inspired characters to customized “friends,” have become ubiquitous—especially among teens craving connection and entertainment online. Yet, as headlines about concerning chatbot interactions multiply and legal battles escalate, Meta’s temporary ban underscor

["**Meta Puts Teen Access to AI Characters on Hold Amid Regulatory Storm and Safety Concerns**\n\nMeta, the social media titan behind Facebook, Instagram, and WhatsApp, is taking the extraordinary step of pausing teen access to its popular AI-powered characters across all its platforms worldwide. The decision, set to be implemented in the coming weeks, arrives at a watershed moment for both Meta and the wider AI industry, as companies grapple with the growing outcry over the safety and psychological risks of conversational bots for younger users.\n\n**A Precaution Amid Intensifying Scrutiny**\n\nFor years, tech companies have cultivated AI chatbots as the next leap in digital engagement. These virtual personalities, ranging from celebrity-inspired characters to customized “friends,” have become ubiquitous—especially among teens craving connection and entertainment online. Yet, as headlines about concerning chatbot interactions multiply and legal battles escalate, Meta’s temporary ban u